In [1]:
# %% [markdown]
"""
# 🔧 Tuning Estrategia MA Envelope Reversals - v3

## Descripción de la estrategia
Detecta oportunidades de reversión cuando el precio alcanza niveles extremos de un canal tipo envelope:
- Canal definido por una media móvil simple y un porcentaje superior/inferior (envelope_pct)
- Confirmación de reversión mediante vela japonesa (cuerpo verde o rojo)
- Filtro opcional por volatilidad (ratio ATR)

## Parámetros configurables
- `window`: int (ventana para la SMA)
- `envelope_pct`: float (ancho relativo del canal)
- `usar_filtro_volatilidad`: bool
- `atr_threshold`: float
- `debug`: bool (no se usa en tuning)

## Objetivo del notebook
Explorar combinaciones de configuración para identificar setups óptimos que generen señales efectivas de reversión en extremos de rango.
"""

# %%
# Carga de historicos desde ruta_historicos
import os
import pandas as pd
from pathlib import Path

ruta_historicos = Path("D:/trading/data/historic")
parquets = list(ruta_historicos.glob("*.parquet"))
historicos = {}

for archivo in parquets:
    simbolo = archivo.stem
    try:
        df = pd.read_parquet(archivo)
        if "fecha" in df.columns:
            df["fecha"] = pd.to_datetime(df["fecha"])
            historicos[simbolo] = df.sort_values("fecha").reset_index(drop=True)
    except Exception as e:
        print(f"Error al cargar {archivo.name}: {e}")

print(f"Se cargaron {len(historicos)} símbolos.")

# %%
# Definicion del grid de parametros
import itertools

param_grid = {
    "window": [14, 20, 30],
    "envelope_pct": [0.02, 0.03, 0.04],
    "usar_filtro_volatilidad": [False, True],
    "atr_threshold": [0.006, 0.008, 0.010]
}

claves = list(param_grid.keys())
combinaciones = list(itertools.product(*param_grid.values()))
print(f"Total de combinaciones a evaluar: {len(combinaciones)}")

# %%
# Funcion de simulacion por combinacion
import sys
sys.path.append("D:/trading")
from my_modules.estrategias.v3 import ma_envelope_reversals_v3

import numpy as np
from my_modules.estrategias.v3 import ma_envelope_reversals_v3

def simular_combinacion(args):
    combinacion, historicos = args
    params = dict(zip(claves, combinacion))
    params["debug"] = False

    resultados = []

    for simbolo, df in historicos.items():
        try:
            df_signals = ma_envelope_reversals_v3.generar_senales(df.copy(), **params)
            df_signals = df_signals[df_signals["signal"] != "hold"]
            if df_signals.empty:
                continue

            df_precio = df.copy()
            df_precio["fecha"] = pd.to_datetime(df_precio["fecha"])
            df_signals["fecha"] = pd.to_datetime(df_signals["fecha"])
            df_merged = df_precio.merge(df_signals, on="fecha")

            for _, row in df_merged.iterrows():
                fecha_entrada = row["fecha"]
                precio_entrada = row["close"]

                df_rango = df_precio[(df_precio["fecha"] > fecha_entrada) &
                                     (df_precio["fecha"] <= fecha_entrada + pd.Timedelta(days=7))]
                if df_rango.empty:
                    continue

                tipo_salida = "TIMEOUT"
                fila_salida = df_rango.iloc[-1]

                for _, f in df_rango.iterrows():
                    if row["signal"] == "buy":
                        if f["high"] >= precio_entrada * 1.05:
                            tipo_salida = "TP"; fila_salida = f; break
                        if f["low"] <= precio_entrada * 0.97:
                            tipo_salida = "SL"; fila_salida = f; break
                    elif row["signal"] == "sell":
                        if f["low"] <= precio_entrada * 0.95:
                            tipo_salida = "TP"; fila_salida = f; break
                        if f["high"] >= precio_entrada * 1.03:
                            tipo_salida = "SL"; fila_salida = f; break

                precio_salida = fila_salida["close"]
                dias = (fila_salida["fecha"] - fecha_entrada).days
                resultado = precio_salida - precio_entrada if row["signal"] == "buy" else precio_entrada - precio_salida
                ret_pct = (precio_salida / precio_entrada - 1) * (1 if row["signal"] == "buy" else -1)
                log_ret = np.log(precio_salida / precio_entrada) * (1 if row["signal"] == "buy" else -1)

                resultados.append({
                    **params,
                    "resultado": resultado,
                    "ret_pct": ret_pct,
                    "log_ret": log_ret,
                    "f_win": int(resultado > 0),
                    "dias": dias
                })

        except Exception as e:
            print(f"[ERROR] {simbolo}: {e}")

    df = pd.DataFrame(resultados)
    if df.empty:
        return {**params, "n_trades": 0, "winrate": 0, "avg_profit": 0, "score": -999}

    return {
        **params,
        "n_trades": len(df),
        "winrate": df["f_win"].mean(),
        "avg_profit": df["resultado"].mean(),
        "score": df["resultado"].mean() * df["f_win"].mean()
    }


# Tuning en paralelo usando joblib + tqdm
from joblib import Parallel, delayed
from tqdm import tqdm

def ejecutar_tuning(historicos, combinaciones):
    def safe_simulacion(comb):
        try:
            return simular_combinacion((comb, historicos))
        except Exception as e:
            print(f"Error en combinación {comb}: {e}")
            return {"params": dict(zip(claves, comb)), "score": -999, "n": 0}

    resultados = Parallel(n_jobs=-1, backend="loky")(
        delayed(safe_simulacion)(comb) for comb in tqdm(combinaciones)
    )
    return resultados

# Ejecutar tuning
print("Ejecutando tuning con joblib...")
resultados = ejecutar_tuning(historicos, combinaciones)

Se cargaron 48 símbolos.
Total de combinaciones a evaluar: 54
Ejecutando tuning con joblib...


100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [04:54<00:00,  5.45s/it]


In [2]:
df_tuning = pd.DataFrame(resultados).sort_values("score", ascending=False).reset_index(drop=True)

print("✅ Top combinaciones por score:")
display(df_tuning.head(10))

✅ Top combinaciones por score:


,window,envelope_pct,usar_filtro_volatilidad,atr_threshold,debug,n_trades,winrate,avg_profit,score
0,14,0.04,False,0.008,False,12574,0.440830,-0.098260,-0.043316
1,14,0.04,True,0.008,False,12574,0.440830,-0.098260,-0.043316
2,14,0.04,False,0.010,False,12574,0.440830,-0.098260,-0.043316
3,14,0.04,True,0.006,False,12574,0.440830,-0.098260,-0.043316
4,14,0.04,False,0.006,False,12574,0.440830,-0.098260,-0.043316
5,14,0.04,True,0.010,False,12573,0.440786,-0.098292,-0.043326
6,30,0.02,True,0.006,False,44303,0.449721,-0.099483,-0.044740
7,30,0.02,False,0.006,False,44305,0.449701,-0.099494,-0.044743
8,30,0.02,False,0.008,False,44305,0.449701,-0.099494,-0.044743
9,30,0.02,False,0.010,False,44305,0.449701,-0.099494,-0.044743
